In [ ]:
from itertools import combinations
from typing import Dict, Tuple, List, Set


# --- Input data (fixed to valid Python tuples for 1-element cases) ---
POKEDEX: Dict[str, Tuple[str, ...]] = {
    "Pikachu": ("Electric",),                      # note the trailing comma → a 1-element tuple
    "Charizard": ("Fire", "Flying"),
    "Lapras": ("Water", "Ice"),
    "Machamp": ("Fighting",),
    "Mewtwo": ("Psychic", "Fighting"),
    "Hoopa": ("Psychic", "Ghost", "Dark"),
    "Lugia": ("Psychic", "Flying", "Water"),
    "Squirtle": ("Water",),
    "Gengar": ("Ghost", "Poison"),
    "Onix": ("Rock", "Ground"),
}


def strongest_teams(
    pokedex: Dict[str, Tuple[str, ...]],
    k: int,
) -> Tuple[int, List[Tuple[Tuple[str, ...], Set[str]]]]:
    """
    Compute every squad of size k, measure the number of unique types each squad covers,
    and return all squads that achieve the maximum coverage (ties allowed).

    Returns:
        max_types: the best (maximum) number of distinct types any k-pokemon squad achieves
        winners:   list of (squad_names_tuple, combined_types_set) for all best squads
    """
    n = len(pokedex)
    if not (1 <= k <= n):
        raise ValueError(f"k must be between 1 and {n}, got {k}")

    items = list(pokedex.items())  # [(name, types_tuple), ...]
    best_strength = -1
    winners: List[Tuple[Tuple[str, ...], Set[str]]] = []

    # Try every k-sized squad
    for squad_items in combinations(items, k):
        # ('Charizard', 'Lapras', 'Hoopa'), for example
        squad_names = tuple(name for name, _ in squad_items)

        # Union of all types present in this squad
        combined_types = {t for _, types in squad_items for t in types}
        strength = len(combined_types)

        # Track only the best; allow ties
        if strength > best_strength:
            best_strength = strength
            winners = [(squad_names, combined_types)]
        elif strength == best_strength:
            winners.append((squad_names, combined_types))

    # Sort just for a stable, nice-looking output
    winners.sort(key=lambda pair: pair[0])
    return best_strength, winners


# --- Example run ---
if __name__ == "__main__":
    k = 3
    best_count, best_squads = strongest_teams(POKEDEX, k)

    print(f"Best number of distinct types for k={k}: {best_count}\n")
    print("Strongest squads (ties shown):")
    for squad_names, types in best_squads:
        names_str = ", ".join(squad_names)
        types_str = ", ".join(sorted(types))
        print(f"- {names_str}  →  {{{types_str}}}")


Best number of distinct types for k=3: 7

Strongest squads (ties shown):
- Charizard, Hoopa, Onix  →  {Dark, Fire, Flying, Ghost, Ground, Psychic, Rock}
- Charizard, Lapras, Hoopa  →  {Dark, Fire, Flying, Ghost, Ice, Psychic, Water}
- Hoopa, Lugia, Onix  →  {Dark, Flying, Ghost, Ground, Psychic, Rock, Water}
- Lapras, Hoopa, Onix  →  {Dark, Ghost, Ground, Ice, Psychic, Rock, Water}
- Lugia, Gengar, Onix  →  {Flying, Ghost, Ground, Poison, Psychic, Rock, Water}


In [ ]:
from collections import Counter


def earliest_step_to_form(word: str, runes: str) -> int:
    """
    Find the earliest position in the rune sequence where the target word
    can be formed (ignoring case and order). Return -1 if impossible.
    """
    target = word.upper()  # normalize target word to uppercase
    target_count = Counter(target)  # required frequency of each letter

    collected = Counter()  # counts of runes collected so far

    for i, rune in enumerate(runes, start=1):  # start=1 to steps are 1-based
        collected[rune.upper()] += 1  # normalize rune to uppercase

        # check if collected runes cover all needed letters
        if all(collected[char] >= target_count[char] for char in target_count):
            return i  # earliest step found

    return -1  # not enough runes in the whole sequence


# --- Example usage ---
if __name__ == "__main__":
    sequence = "LxUMyzOoSsM"  # Hermione pulls out runes in this order
    step = earliest_step_to_form("LUMOS", sequence)
    print(f"Earliest step: {step}")


Earliest step: 9


In [ ]:
def max_damage_bomb(grid, m):
    n = len(grid)  # size of the map (n x n)
    radius = m // 2  # how far the blast extends in each direction from the center

    max_damage = 0
    best_coord = None
    destroyed_coords = []

    # iterate over every possible cell as the bomb center
    for i in range(n):
        for j in range(n):
            if grid[i][j] == 0:
                continue  # bomb won't detonate on water

            damage = 0
            current_destroyed = []

            # check m x m region around (i, j)
            for di in range(-radius, radius + 1):
                for dj in range(-radius, radius + 1):
                    ni, nj = i + di, j + dj
                    if 0 <= ni < n and 0 <= nj < n and grid[ni][nj] == 1:
                        damage += 1
                        # convert matrix index to coordinate (x=j, y=i)
                        current_destroyed.append((nj, i))

            # update max if better
            if damage > max_damage:
                max_damage = damage
                best_coord = (j, i)  # coordinate system (x=j, y=i)
                destroyed_coords = current_destroyed

    return best_coord, max_damage, destroyed_coords


# --- Example usage ---
if __name__ == "__main__":
    grid = [
        [1, 0, 0, 0, 1],
        [1, 0, 1, 1, 1],
        [1, 1, 0, 1, 1],
        [1, 0, 1, 1, 0],
        [0, 1, 0, 1, 1]
    ]
    m = 3
    coord, damage, destroyed = max_damage_bomb(grid, m)

    print(f"Best bomb location: {coord}")
    print(f"Max islands destroyed: {damage}")
    print("Destroyed coordinates:", destroyed)


Best bomb location: (3, 2)
Max islands destroyed: 7
Destroyed coordinates: [(2, 2), (3, 2), (4, 2), (3, 2), (4, 2), (2, 2), (3, 2)]


In [2]:
# ===========================================
# Exam Oracle 3000 — Forward Propagation (NumPy only)
# Architecture: 3 -> 4 -> 2, sigmoid on all neurons
# Three different oracles (v1, v2, v3) with different manual weights/biases
# ===========================================

import numpy as np

# 1) Sigmoid activation (σ(z) = 1 / (1 + e^{-z}))
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

# 2) Forward pass for a single hidden layer MLP
#    Shapes:
#    - X: (3, 1) column vector [study_hours, attendance_pct, quiz_score]
#    - W1: (4, 3), b1: (4, 1)  -> hidden layer
#    - W2: (2, 4), b2: (2, 1)  -> output layer (Pass, Fail)
def forward(X, W1, b1, W2, b2):
    Z1 = np.dot(W1, X) + b1   # (4,1)
    A1 = sigmoid(Z1)          # (4,1)
    Z2 = np.dot(W2, A1) + b2  # (2,1)
    A2 = sigmoid(Z2)          # (2,1) -> [Pass_prob, Fail_prob]
    return Z1, A1, Z2, A2

# 3) Define three student profiles (columns are separate runs)
students = {
    "A": np.array([[ 8.0], [90.0], [85.0]]),  # strong -> should lean Pass
    "B": np.array([[ 2.0], [30.0], [20.0]]),  # weak   -> likely Fail
    "C": np.array([[ 5.0], [60.0], [55.0]]),  # mixed  -> mystery case
}

# 4) Hidden layer weights/biases (shared across all oracles for clarity)
#    4 hidden neurons. Each row = one neuron’s weights over 3 inputs.
#    We keep attendance & quiz weights much smaller since they can be ~0..100.
W1_shared = np.array([
    [0.60, 0.02, 0.04],  # h1: hours-focused, slight attendance/quiz
    [0.20, 0.01, 0.08],  # h2: quiz-focused with small hours/attendance
    [0.05, 0.05, 0.01],  # h3: attendance-focused
    [0.10, 0.02, 0.02],  # h4: "balanced" synergy unit
], dtype=float)

b1_shared = np.array([
    [-2.0],  # h1 threshold
    [-5.0],  # h2 stricter threshold
    [-4.0],  # h3 attendance threshold
    [-1.0],  # h4 mild threshold
], dtype=float)

# 5) Output layer (Pass, Fail) — three different "oracle" heads.
#    TIP: we make the Fail neuron the negative mirror of Pass (weights & bias)
#    so that Fail ≈ 1 - Pass because σ(-z) = 1 - σ(z).
def make_head(pass_weights_row, pass_bias):
    W2_pass = np.array([pass_weights_row], dtype=float)   # (1,4)
    W2_fail = -W2_pass                                    # (1,4)
    W2 = np.vstack([W2_pass, W2_fail])                    # (2,4)
    b2 = np.vstack([[[pass_bias]], [[-pass_bias]]])       # (2,1)
    return W2, b2

# Oracle v1: balanced; strong emphasis on hours & quiz; moderate bias
W2_v1, b2_v1 = make_head([3.0, 2.0, 1.5, 1.0], pass_bias=-4.0)

# Oracle v2: attendance-heavy decision (weights favor hidden unit 3)
W2_v2, b2_v2 = make_head([1.0, 0.5, 4.0, 0.5], pass_bias=-2.5)

# Oracle v3: quiz-heavy decision (weights favor hidden unit 2)
W2_v3, b2_v3 = make_head([0.5, 4.0, 0.2, 0.2], pass_bias=-2.0)

oracles = [
    ("Oracle v1 (balanced)",       W1_shared, b1_shared, W2_v1, b2_v1),
    ("Oracle v2 (attendance-bias)", W1_shared, b1_shared, W2_v2, b2_v2),
    ("Oracle v3 (quiz-bias)",       W1_shared, b1_shared, W2_v3, b2_v3),
]

# 6) Helper for dramatic verdicts
def verdict(pass_prob):
    p = float(pass_prob)
    pct = int(round(p * 100))
    if p >= 0.95:
        return f"🔮 The Oracle whispers: {pct}% chance of Passing. Rejoice, mortal!"
    elif p >= 0.70:
        return f"✨ The Oracle smiles: {pct}% chance of Passing. Promising signs!"
    elif p >= 0.40:
        return f"🤔 The Oracle muses: {pct}% chance of Passing. It could go either way…"
    else:
        return f"💀 The Oracle warns: {pct}% chance of Passing. May the curve be ever in your favor…"

# 7) Run all oracles on all students, printing Z and A at each layer
comparison = {}  # for quick cross-oracle summary later

for name, W1, b1, W2, b2 in oracles:
    print("\n" + "="*60)
    print(name)
    print("="*60)
    comparison[name] = {}
    for label, X in students.items():
        print(f"\nStudent {label} profile [hours, attendance, quiz]: {X.ravel().tolist()}")
        Z1, A1, Z2, A2 = forward(X, W1, b1, W2, b2)

        # Display internals for pen-and-paper verification
        print("Z1 (hidden pre-activations):\n", np.round(Z1, 3))
        print("A1 (hidden activations σ(Z1)):\n", np.round(A1, 3))
        print("Z2 (output pre-activations):\n", np.round(Z2, 3))

        # Final prediction: [Pass, Fail]
        probs = A2.flatten()
        print("Oracle prediction (Pass, Fail):", np.round(probs, 3))
        print(verdict(probs[0]))

        # Store for comparison table
        comparison[name][label] = probs[0]  # Pass probability

# 8) Quick comparison table of Pass% across oracles
print("\n" + "="*60)
print("Quick Comparison (Pass % by Oracle)")
print("="*60)
hdr = "Student   " + "   ".join(n.split(" (")[0] for n, *_ in oracles)
print(hdr)
for s in students.keys():
    row = [s]
    for n, *_ in oracles:
        row.append(f"{int(round(comparison[n][s]*100)):3d}%")
    print(("   ").join(row))



Oracle v1 (balanced)

Student A profile [hours, attendance, quiz]: [8.0, 90.0, 85.0]
Z1 (hidden pre-activations):
 [[8.  ]
 [4.3 ]
 [1.75]
 [3.3 ]]
A1 (hidden activations σ(Z1)):
 [[1.   ]
 [0.987]
 [0.852]
 [0.964]]
Z2 (output pre-activations):
 [[ 3.215]
 [-3.215]]
Oracle prediction (Pass, Fail): [0.961 0.039]
🔮 The Oracle whispers: 96% chance of Passing. Rejoice, mortal!

Student B profile [hours, attendance, quiz]: [2.0, 30.0, 20.0]
Z1 (hidden pre-activations):
 [[ 0.6]
 [-2.7]
 [-2.2]
 [ 0.2]]
A1 (hidden activations σ(Z1)):
 [[0.646]
 [0.063]
 [0.1  ]
 [0.55 ]]
Z2 (output pre-activations):
 [[-1.238]
 [ 1.238]]
Oracle prediction (Pass, Fail): [0.225 0.775]
💀 The Oracle warns: 22% chance of Passing. May the curve be ever in your favor…

Student C profile [hours, attendance, quiz]: [5.0, 60.0, 55.0]
Z1 (hidden pre-activations):
 [[ 4.4]
 [ 1. ]
 [-0.2]
 [ 1.8]]
A1 (hidden activations σ(Z1)):
 [[0.988]
 [0.731]
 [0.45 ]
 [0.858]]
Z2 (output pre-activations):
 [[ 1.959]
 [-1.959]]
Or